In [1]:
from tensorflow.keras.datasets import mnist #케라스(tensorflow >2.0)에서 받을수 있는 데이터 셋
from tensorflow.python.keras.utils.np_utils import to_categorical #np_utils 모듈 수정됨, 
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D #Dropout, Flatten, Conv2D 등 import
from tensorflow.keras.callbacks import ModelCheckpoint,EarlyStopping #모델저장, 과적합 방지
from PyQt5.QtWidgets import QApplication, QWidget, QLabel, QVBoxLayout, QPushButton, QMessageBox
from PyQt5 import QtGui
from PyQt5 import QtCore


import matplotlib.pyplot as plt
import numpy as np
import threading
import sys
import os
import tensorflow as tf
import cv2

running = False

In [2]:
class MyApp(QWidget):

    def __init__(self):
        super().__init__()
        self.initUI()
        self.initMODEL()
    
    def initMODEL(self):
        (X_train, Y_train), (X_test, Y_test) = mnist.load_data()
        X_train = X_train.reshape(X_train.shape[0], 28, 28, 1).astype('float32') / 255
        X_test = X_test.reshape(X_test.shape[0], 28, 28, 1).astype('float32') / 255
        Y_train = to_categorical(Y_train) 
        Y_test = to_categorical(Y_test)
        
        self.model = load_model('./dataset/MNIST_학습된 모델/09-0.0266.hdf5') # 모델을 새로 불러옴

        print("\n Test Accuracy: %.4f" % (self.model.evaluate(X_test, Y_test)[1]))  # 불러온 모델로 테스트 실행
    
    def initUI(self):
        self.cap = cv2.VideoCapture(0)
        frmae_size = (int(self.cap.get(cv2.CAP_PROP_FRAME_WIDTH)),
                      int(self.cap.get(cv2.CAP_PROP_FRAME_HEIGHT)))
        
        self.cam_label = QLabel()
        self.cam_label.resize(frmae_size[0], frmae_size[1])
        
        self.recog_label = QLabel()

        self.start_cam_btn = QPushButton('CAM ON', self)
        self.start_cam_btn.clicked.connect(self.CAM_ON)
        self.start_cam_btn.setEnabled(True)

        self.stop_cam_btn = QPushButton('CAM OFF', self)
        self.stop_cam_btn.clicked.connect(self.CAM_OFF)
        self.stop_cam_btn.setEnabled(False)
        
        self.capture = QPushButton('숫자 인식', self)
        self.capture.clicked.connect(self.CAPTURE)
        self.capture.setEnabled(False)

        app.aboutToQuit.connect(self.onExit)

        vbox = QVBoxLayout()
        vbox.addWidget(self.cam_label)
        vbox.addWidget(self.start_cam_btn)
        vbox.addWidget(self.stop_cam_btn)
        vbox.addWidget(self.capture)
        vbox.addWidget(self.recog_label)
        

        self.setLayout(vbox)
        self.setWindowTitle('PC_CAM')

        self.resize(600, 480)
        self.show()

    def RUN(self):
        global running

        while running:
            ret, self.frame = self.cap.read()
            if ret:
                self.frame = cv2.cvtColor(self.frame, cv2.COLOR_BGR2RGB)
                h,w,c = self.frame.shape
                qimg = QtGui.QImage(self.frame.data, w, h, w*c, QtGui.QImage.Format_RGB888)
                pixmap = QtGui.QPixmap.fromImage(qimg)
                self.cam_label.setPixmap(pixmap)
            else:
                QMessageBox.about(win, "Error", "Cannot read frame.")
                print("error")
                break
        self.cap.release()
        print('Thread end.')

    def CAM_ON(self):
        global running
        running = True
        th = threading.Thread(target=self.RUN)
        th.start()
        print("start")
        self.start_cam_btn.setEnabled(False)
        self.stop_cam_btn.setEnabled(True)
        self.capture.setEnabled(True)

    def CAM_OFF(self):
        global running
        running = False
        print("stop")
        self.start_cam_btn.setEnabled(True)
        self.stop_cam_btn.setEnabled(False)
        self.capture.setEnabled(False)
        
    def CAPTURE(self):

        cv2.imwrite('./num_cap.png', self.frame)
        
        src = cv2.imread('./num_cap.png', cv2.IMREAD_GRAYSCALE)

        ret, blmage = cv2.threshold(src, 127, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

        kernel = cv2.getStructuringElement(shape=cv2.MORPH_RECT, ksize=(15,15))
        erode = cv2.erode(blmage, kernel, 7)

        dst = erode[100:450, 100:500].copy()

        resize_img = cv2.resize(dst, (28, 28), fx=1, fy=1,interpolation = cv2.INTER_AREA)
        last_img = 255 - resize_img.copy()

        img3 = last_img.reshape(1, 28, 28, 1) / 255
        
        text = self.model.predict_classes(img3)
        
        self.RECOGNITION(text)
        
    def RECOGNITION(self, text):
        self.recog_label.setText(str(text[0]))
        self.recog_label.adjustSize()
        

    def onExit(self):
        global running
        running = False
        print("exit")

if __name__ == '__main__':
    app = QApplication(sys.argv)
    ex = MyApp()
    sys.exit(app.exec_())

10000/10000 [==============================] - 4s 445us/sample - loss: 0.0266 - accuracy: 0.9915

 Test Accuracy: 0.9915
start
exit


SystemExit: 0

C:\Users\user\anaconda3\envs\edge1\lib\site-packages\IPython\core\interactiveshell.py:3445: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


Thread end.
